In [1]:
import sys
import random
import numpy as np
import os
from PIL import Image
from mujoco_env.y_env2 import SimpleEnv2
from lerobot.common.datasets.lerobot_dataset import LeRobotDataset

/home/jeongeun/.pyenv/versions/3.10.2/envs/lerobot/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# If you want to randomize the object positions, set this to None
# If you fix the seed, the object positions will be the same every time
SEED = 0 
# SEED = None <- Uncomment this line to randomize the object positions

REPO_NAME = 'omy_pnp'
NUM_DEMO = 50 # Number of demonstrations to collect
ROOT = "./omy_pnp" # The root directory to save the demonstrations

In [3]:
xml_path = './asset/example_scene_y2.xml'
# Define the environment
PnPEnv = SimpleEnv2(xml_path, seed = SEED, state_type = 'joint_angle')


-----------------------------------------------------------------------------
name:[Tabletop] dt:[0.002] HZ:[500]
 n_qpos:[31] n_qvel:[28] n_qacc:[28] n_ctrl:[10]
 integrator:[IMPLICITFAST]

n_body:[23]
 [0/23] [world] mass:[0.00]kg
 [1/23] [front_object_table] mass:[1.00]kg
 [2/23] [camera] mass:[0.00]kg
 [3/23] [camera2] mass:[0.00]kg
 [4/23] [camera3] mass:[0.00]kg
 [5/23] [link1] mass:[2.06]kg
 [6/23] [link2] mass:[3.68]kg
 [7/23] [link3] mass:[2.39]kg
 [8/23] [link4] mass:[1.40]kg
 [9/23] [link5] mass:[1.40]kg
 [10/23] [link6] mass:[0.65]kg
 [11/23] [camera_center] mass:[0.00]kg
 [12/23] [tcp_link] mass:[0.32]kg
 [13/23] [rh_p12_rn_r1] mass:[0.07]kg
 [14/23] [rh_p12_rn_r2] mass:[0.02]kg
 [15/23] [rh_p12_rn_l1] mass:[0.07]kg
 [16/23] [rh_p12_rn_l2] mass:[0.02]kg
 [17/23] [body_obj_mug_5] mass:[0.00]kg
 [18/23] [object_mug_5] mass:[0.08]kg
 [19/23] [body_obj_plate_11] mass:[0.00]kg
 [20/23] [object_plate_11] mass:[0.10]kg
 [21/23] [body_obj_mug_6] mass:[0.00]kg
 [22/23] [object_mug

In [4]:
create_new = True
if os.path.exists(ROOT):
    print(f"Directory {ROOT} already exists.")
    ans = input("Do you want to delete it? (y/n) ")
    if ans == 'y':
        import shutil
        shutil.rmtree(ROOT)
    else:
        create_new = False


if create_new:
    dataset = LeRobotDataset.create(
                repo_id=REPO_NAME,
                root = ROOT, 
                robot_type="omy",
                fps=20, # 20 frames per second
                features={
                    "observation.image": {
                        "dtype": "image",
                        "shape": (256, 256, 3),
                        "names": ["height", "width", "channels"],
                    },
                    "observation.wrist_image": {
                        "dtype": "image",
                        "shape": (256, 256, 3),
                        "names": ["height", "width", "channel"],
                    },
                    "observation.state": {
                        "dtype": "float32",
                        "shape": (6,),
                        "names": ["state"], # x, y, z, roll, pitch, yaw
                    },
                    "action": {
                        "dtype": "float32",
                        "shape": (7,),
                        "names": ["action"], # 6 joint angles and 1 gripper
                    },
                    "obj_init": {
                        "dtype": "float32",
                        "shape": (9,),
                        "names": ["obj_init"], # just the initial position of the object. Not used in training.
                    },
                },
                image_writer_threads=10,
                image_writer_processes=5,
        )
else:
    print("Load from previous dataset")
    dataset = LeRobotDataset(REPO_NAME, root=ROOT)

Directory ./demo_data_language already exists.


Process Process-1:
Process Process-3:
Process Process-4:
Process Process-2:
Process Process-5:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/jeongeun/.pyenv/versions/3.10.2/lib/python3.10/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/jeongeun/.pyenv/versions/3.10.2/lib/python3.10/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/jeongeun/.pyenv/versions/3.10.2/lib/python3.10/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/jeongeun/.pyenv/versions/3.10.2/lib/python3.10/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/jeongeun/.pyenv/versions/3.10.2/lib/python3.10/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/jeongeun/.pyenv/versions/3.10.2/lib/python3.10/multiprocessing/process.

In [5]:
action = np.zeros(7)
episode_id = 0
record_flag = False # Start recording when the robot starts moving
while PnPEnv.env.is_viewer_alive() and episode_id < NUM_DEMO:
    PnPEnv.step_env()
    if PnPEnv.env.loop_every(HZ=20):
        # check if the episode is done
        done = PnPEnv.check_success()
        if done: 
            # Save the episode data and reset the environment
            dataset.save_episode()
            PnPEnv.reset()
            episode_id += 1
        # Teleoperate the robot and get delta end-effector pose with gripper
        action, reset  = PnPEnv.teleop_robot()
        if not record_flag and sum(action) != 0:
            record_flag = True
            print("Start recording")
        if reset:
            # Reset the environment and clear the episode buffer
            # This can be done by pressing 'z' key
            # PnPEnv.reset(seed=SEED)
            PnPEnv.reset()
            dataset.clear_episode_buffer()
            record_flag = False
        # Step the environment
        joint_q = PnPEnv.step(action)
        # Get the end-effector pose and images
        ee_pose = PnPEnv.get_ee_pose()
        agent_image,wrist_image = PnPEnv.grab_image()
        # # resize to 256x256
        agent_image = Image.fromarray(agent_image)
        wrist_image = Image.fromarray(wrist_image)
        agent_image = agent_image.resize((256, 256))
        wrist_image = wrist_image.resize((256, 256))
        agent_image = np.array(agent_image)
        wrist_image = np.array(wrist_image)
        if record_flag:
            # Add the frame to the dataset
            dataset.add_frame( {
                    "observation.image": agent_image,
                    "observation.wrist_image": wrist_image,
                    "observation.state": ee_pose, 
                    "action": joint_q,
                    "obj_init": PnPEnv.obj_init_pose,
                    # "task": PnPEnv.instruction,
                }, task = PnPEnv.instruction
            )
        PnPEnv.render(teleop=True, idx=episode_id)

Start recording


Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 65.01ba/s]


DONE INITIALIZATION


Creating parquet from Arrow format: 100%|██████████| 2/2 [00:00<00:00, 93.40ba/s]


DONE INITIALIZATION
DONE INITIALIZATION
Start recording


...

DONE INITIALIZATION


Creating parquet from Arrow format: 100%|██████████| 2/2 [00:00<00:00, 110.75ba/s]


DONE INITIALIZATION


Creating parquet from Arrow format: 100%|██████████| 2/2 [00:00<00:00, 102.26ba/s]


DONE INITIALIZATION


In [6]:
PnPEnv.env.close_viewer()

In [1]:
# Clean up the images folder
import shutil
shutil.rmtree(dataset.root / 'images')

NameError: name 'dataset' is not defined